# Chained Pipeline: ABAP Code Data Type Modernization

This notebook executes a pipeline for ABAP code remediation:

1. **Scan Program**: Extracts SELECT queries, DATA TYPEs, and FUNCTION models using `Scan-Program.ipynb` logic.
2. **Transform DATA TYPEs**: For each found DATA TYPE, uses the mapping in `Transform-DE-DT.ipynb` to get its S4 declaration. The code is then rewritten, replacing every original DATA TYPE with the mapped S4 declaration.

## Requirements
- Place `Scan-Program.ipynb`, `Transform-DE-DT.ipynb`, and `Mapping-DE-DT.xlsx` in the same folder.
- Set `input_code_file` to the path of the ABAP code file to process (or supply the code as a list of lines).
- The output is a transformed version of the ABAP code, printed as text (and optionally written to a file).


In [ ]:
# Import scanner and transformer code from the supplied notebooks
# For a real pipeline, you'd use %run or refactor the code into modules. Here, we inline the logic for clarity.
import re
import pandas as pd


## 1. Load and Scan the ABAP Code

In [ ]:
# --- Load your ABAP program ---
input_code_file = 'sample_program.abap'  # Change this to your ABAP code file path
with open(input_code_file, encoding='utf-8') as f:
    code_lines = [line.rstrip('\n') for line in f]

# --- Scanner logic from Scan-Program.ipynb ---
def scan_keywords(code_lines):
    select_queries = []
    data_types = []
    function_names = []
    in_select = False
    current_select = []
    for line in code_lines:
        l = line.rstrip()
        if not in_select and re.search(r'\bSELECT\b', l, re.IGNORECASE):
            in_select = True
            current_select = [l]
            if l.strip().endswith(('.', ';')):
                select_queries.append(' '.join(current_select))
                in_select = False
                current_select = []
            continue
        if in_select:
            current_select.append(l)
            if l.strip().endswith(('.', ';')):
                select_queries.append(' '.join(current_select))
                in_select = False
                current_select = []
            continue
        if re.search(r'\bDATA\b', l, re.IGNORECASE):
            match = re.search(r'TYPE\s+([\w\.\_]+)', l, re.IGNORECASE)
            if match:
                data_types.append(match.group(1))
        if re.search(r'\bCALL FUNCTION\b', l, re.IGNORECASE):
            match = re.search(r'CALL FUNCTION\s+["\']?([\w\_]+)["\']?', l, re.IGNORECASE)
            if match:
                function_names.append(match.group(1))
    return select_queries, data_types, function_names

selects, data_types, functions = scan_keywords(code_lines)

print('SELECT Queries:')
for s in selects:
    print('---')
    print(s)

print('\nDATA TYPEs:')
for d in data_types:
    print(' ', d)

print('\nFUNCTION Names:')
for f in functions:
    print(' ', f)


## 2. Load the Mapping Sheet and Build the Data Type Transformation Map

In [ ]:
mapping_file = 'Mapping-DE-DT.xlsx'  # Should be in the same folder
df = pd.read_excel(mapping_file)

def get_s4_declaration(data_element):
    row = df.loc[df['CRM'] == data_element]
    if row.empty:
        return None  # No mapping found
    s4 = str(row.iloc[0]['S4']).strip()
    length = row.iloc[0]['LENGTH'] if 'LENGTH' in row.columns else None
    decimal = row.iloc[0]['DECIMAL'] if 'DECIMAL' in row.columns else None
    result = s4
    if pd.notnull(length):
        result += f" LENGTH {int(float(length))}"
    if decimal is not None and pd.notnull(decimal):
        result += f" DECIMALS {int(float(decimal))}"
    return result

# Build mapping for all detected DATA TYPEs
type_transform_map = {}
for dtype in set(data_types):
    mapped = get_s4_declaration(dtype)
    if mapped:
        type_transform_map[dtype] = mapped

print('Data Element Mappings:')
for k, v in type_transform_map.items():
    print(f'  {k} -> {v}')


## 3. Replace DATA TYPEs in Code Using the Discovered Mappings

In [ ]:
def replace_data_types_in_line(line, type_map):
    # Replace only the TYPE <type> part in a DATA line
    # Handles e.g. DATA lv_name TYPE string.
    for orig, repl in type_map.items():
        # Use regex with word boundary to avoid partial matches
        # Accepts 'TYPE xxx' where xxx is the original type
        pattern = rf'(TYPE\s+){orig}\b'
        if re.search(pattern, line, flags=re.IGNORECASE):
            # Only replace the type, keep rest of line (e.g. ending dot)
            line = re.sub(pattern, f"\\1{repl}", line, flags=re.IGNORECASE)
    return line

transformed_lines = []
for line in code_lines:
    transformed_lines.append(replace_data_types_in_line(line, type_transform_map))

# Optionally, write output to a new file
output_file = input_code_file.replace('.abap', '_S4.abap')
with open(output_file, 'w', encoding='utf-8') as f:
    for line in transformed_lines:
        f.write(line + '\n')

# Preview the transformed code
print('\n===== Transformed ABAP Program =====\n')
for line in transformed_lines:
    print(line)
